In [62]:
# Cargamos algunas librerías necesarias.
import numpy  as np  
import pandas as pd
pd.set_option('display.max_columns', 100) # Con esto garantizo el ver todas las columnas.

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt # Para dibujar alguna gráfica.
%matplotlib inline

from geopy.geocoders import Nominatim
import geocoder
import requests
import re
import threading 
import time
from math import sin, cos, sqrt, atan2, radians

In [63]:
df = pd.read_csv('data/Accidentes.csv',sep=';', decimal='.')

In [64]:
df.head()

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592
1,1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.398454,-3.601018
2,2,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS,40.393845,-3.680282
3,3,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS,40.377139,-3.739303
4,4,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS,40.410009,-3.641304


In [65]:
#cargamos el data frame de las cordenadas de ocio
df_ocio = pd.read_csv('data/Coordenadas_ocio.csv',sep=';', decimal='.')

In [66]:
df_ocio.head()

,Latidud_ocio,Longitud_ocio
0,40.565647,-3.913978
1,40.055614,-3.604527
2,40.424492,-3.691446
3,40.420115,-3.700815
4,40.453986,-3.694140


In [67]:
# calculamos las distancias
def distancia_ocio(long1,lat1,long2,lat2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(long1)
    lat2 = radians(lat2)
    lon2 = radians(long2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [68]:
# nos quedamos con la que esta mas cerca
def comparador(row):
    lat = row['Latitude']
    long = row['Longitude']
    v = list()
    for i in range(len(df_ocio)):
        c = distancia_ocio(df_ocio['Longitud_ocio'][i],df_ocio['Latidud_ocio'][i], long, lat )
        v.append(c)
    return min(v)

In [69]:
# creamos la columna con las distancias mas cercanas
df['Distancia ocio Km'] = df[['Latitude','Longitude']].apply(lambda row: comparador(row), axis=1)

In [70]:
df.head()

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801
1,1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.398454,-3.601018,2.790873
2,2,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS,40.393845,-3.680282,0.948289
3,3,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS,40.377139,-3.739303,0.984928
4,4,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS,40.410009,-3.641304,1.041850


In [71]:
# creamos la columna lejania ocio para saber si esta cerca o lejos
df['Lejania ocio']=''
for x in range(len(df)):
    if df['Distancia ocio Km'][x]<0.2:
        df['Lejania ocio'][x]='Muy cerca'
    elif df['Distancia ocio Km'][x]>=0.2 and df['Distancia ocio Km'][x]<0.5:
        df['Lejania ocio'][x]='Cerca'
    elif df['Distancia ocio Km'][x]>=0.5 and df['Distancia ocio Km'][x]<1:
        df['Lejania ocio'][x]='lejos'
    else:
        df['Lejania ocio'][x]='muy lejos'
        
    

C:\Users\juanc\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\juanc\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\juanc\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\

In [72]:
df.head()

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km,Lejania ocio
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos
1,1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.398454,-3.601018,2.790873,muy lejos
2,2,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS,40.393845,-3.680282,0.948289,lejos
3,3,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS,40.377139,-3.739303,0.984928,lejos
4,4,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS,40.410009,-3.641304,1.041850,muy lejos


In [73]:
# cargamos el dataset de los radares
df_radares = pd.read_csv('data/Radares_Madrid.csv',sep=';',decimal='.', encoding = 'unicode_escape')

In [74]:
df_radares.head()

,Nº\nRADAR,Ubicacion,Carretara o vial,UBICACIÓN\nCalle 30,PK,Sentido,Tipo,X (WGS84),Y (WGS84),Longitud,Latitud,Coordenadas
0,1,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",M-30,00NC50,0+500,Calzada interior,Fijo carriles 3 y 4,442843.00,4481180.00,-367.433.808,4.047.934.148,DGP
1,2,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",M-30,00NC50,0+500,Calzada interior,Fijo carriles 3 y 4,442847.00,4481180.00,-367.429.089,4.047.934.175,DGP
2,3,"PASO INFERIOR COSTA RICA, SENTIDO PRÍNCIPE DE ...",NaN,NaN,NaN,NaN,NaN,443295.00,4478910.00,-366.880.283,4.045.892.328,DGP
3,4,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",M-30,M-30,6+700,Tronco calzada interior,"Fijo carriles 1, 2 y 3",444002.00,4475270.00,-366.014.384,4.042.618.023,DGP
4,5,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",M-30,M-30,6+700,Tronco calzada interior,"Fijo carriles 1, 2 y 3",444009.00,4475271.00,-366.006.141,4.042.618.971,DGP


In [75]:
# nos quedamos con las columnas 1,9,10
df_radares=df_radares.iloc[:,[1,9,10]]

In [76]:
df_radares.head()

,Ubicacion,Longitud,Latitud
0,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",-367.433.808,4.047.934.148
1,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",-367.429.089,4.047.934.175
2,"PASO INFERIOR COSTA RICA, SENTIDO PRÍNCIPE DE ...",-366.880.283,4.045.892.328
3,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",-366.014.384,4.042.618.023
4,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",-366.006.141,4.042.618.971


In [77]:
df_radares['Longitud'].dtype

dtype('O')

In [78]:
df_radares['Longitud']=df_radares['Longitud'].astype('string')

In [79]:
df_radares['Longitud']

0             -367.433.808
1             -367.429.089
2             -366.880.283
3             -366.014.384
4             -366.006.141
5             -365.926.082
6             -365.929.627
7               -3.670.048
8               -3.669.973
9             -369.499.826
10              -3.699.691
11              -3.703.274
12            -371.017.703
13              -3.708.108
14              -3.709.684
15              -3.722.988
16              -3.721.414
17              -3.731.092
18                -373.102
19            -374.545.156
20              -3.727.663
21              -3.727.724
22            -373.482.875
23            -374.530.618
24            -373.089.695
25            -368.542.793
26              -3.696.519
27    -376.358.611.606.955
28    -374.574.755.307.117
Name: Longitud, dtype: string

In [80]:
df_radares['Longitud'].dtype

StringDtype

In [81]:
df_radares['Longitud'] = df_radares['Longitud'].str.replace('.', '')
df_radares['Latitud'] = df_radares['Latitud'].str.replace('.', '')

In [82]:
df_radares.head()

,Ubicacion,Longitud,Latitud
0,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",-367433808,4047934148
1,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",-367429089,4047934175
2,"PASO INFERIOR COSTA RICA, SENTIDO PRÍNCIPE DE ...",-366880283,4045892328
3,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",-366014384,4042618023
4,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",-366006141,4042618971


In [83]:
# Cambiamos los datos a numéricos
df_radares['Longitud'] = df_radares['Longitud'].apply(pd.to_numeric, errors='coerce')
df_radares['Latitud'] = df_radares['Latitud'].apply(pd.to_numeric, errors='coerce')
df_radares.dtypes

Ubicacion    object
Longitud      int64
Latitud       int64
dtype: object

In [84]:
# Movemos la coma para transformarla en el formato de Longitud
df_radares['Longitude'] = df_radares['Longitud'].apply(lambda x: x / 10 ** (len((str(x))) - 2))

In [85]:
# Movemos la coma para transformarla en el formato de Loatitud
df_radares['Latitude'] = df_radares['Latitud'].apply(lambda x: x / 10 ** (len((str(x))) - 2))

In [86]:
df_radares.head()

,Ubicacion,Longitud,Latitud,Longitude,Latitude
0,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",-367433808,4047934148,-3.674338,40.479341
1,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",-367429089,4047934175,-3.674291,40.479342
2,"PASO INFERIOR COSTA RICA, SENTIDO PRÍNCIPE DE ...",-366880283,4045892328,-3.668803,40.458923
3,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",-366014384,4042618023,-3.660144,40.426180
4,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",-366006141,4042618971,-3.660061,40.426190


In [87]:
df_radares.drop(['Longitud', 'Latitud'], axis = 'columns', inplace=True)

In [88]:
df_radares.head()

,Ubicacion,Longitude,Latitude
0,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",-3.674338,40.479341
1,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",-3.674291,40.479342
2,"PASO INFERIOR COSTA RICA, SENTIDO PRÍNCIPE DE ...",-3.668803,40.458923
3,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",-3.660144,40.426180
4,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",-3.660061,40.426190


In [89]:
# calculamos las distancias
def distancia(long1,lat1,long2,lat2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(long1)
    lat2 = radians(lat2)
    lon2 = radians(long2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [90]:
# nos quedamos con la distancia mas cercana
def comparador(row):
    lat = row['Latitude']
    long = row['Longitude']
    v = list()
    for i in range(len(df_radares)):
        c = distancia(df_radares['Longitude'][i],df_radares['Latitude'][i], long, lat )
        v.append(c)
    return min(v)

In [91]:
# creamos la columna con las distancias mas cercanas
df['Distancia_radares'] = df[['Latitude','Longitude']].apply(lambda row: comparador(row), axis=1)

In [92]:
df.head()

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km,Lejania ocio,Distancia_radares
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos,0.507497
1,1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.398454,-3.601018,2.790873,muy lejos,5.331349
2,2,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS,40.393845,-3.680282,0.948289,lejos,0.950698
3,3,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS,40.377139,-3.739303,0.984928,lejos,2.881995
4,4,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS,40.410009,-3.641304,1.041850,muy lejos,1.689948


In [93]:
# creamos la columna lejania radares para saber si esta cerca o lejos
df['Lejania_radares']=''
for x in range(len(df)):
    if df['Distancia_radares'][x]<0.2:
        df['Lejania_radares'][x]='Muy cerca'
    elif df['Distancia_radares'][x]>=0.2 and df['Distancia_radares'][x]<0.5:
        df['Lejania_radares'][x]='Cerca'
    elif df['Distancia_radares'][x]>=0.5 and df['Distancia_radares'][x]<1:
        df['Lejania_radares'][x]='lejos'
    else:
        df['Lejania_radares'][x]='muy lejos'

C:\Users\juanc\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\juanc\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\juanc\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Us

In [94]:
df.head()

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km,Lejania ocio,Distancia_radares,Lejania_radares
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos,0.507497,lejos
1,1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.398454,-3.601018,2.790873,muy lejos,5.331349,muy lejos
2,2,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS,40.393845,-3.680282,0.948289,lejos,0.950698,lejos
3,3,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS,40.377139,-3.739303,0.984928,lejos,2.881995,muy lejos
4,4,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS,40.410009,-3.641304,1.041850,muy lejos,1.689948,muy lejos


In [95]:
df['FECHA_CON_FORMATO'] = df['FECHA'].apply(pd.to_datetime)

In [96]:
# Agrego columnas con dia, mes y año para tener esos valores separados
df['AÑO'] = pd.DatetimeIndex(df['FECHA_CON_FORMATO']).year
df['MES'] = pd.DatetimeIndex(df['FECHA_CON_FORMATO']).month
df['DIA'] = pd.DatetimeIndex(df['FECHA_CON_FORMATO']).day

In [97]:
df.head()

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km,Lejania ocio,Distancia_radares,Lejania_radares,FECHA_CON_FORMATO,AÑO,MES,DIA
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos,0.507497,lejos,2014-11-20,2014,11,20
1,1,04/03/2014,DE 13:00 A 13:59,MARTES,VICALVARO,AVENIDA DE LA GRAN VIA DEL ESTE - CALLE DE LA ...,0,2014/2147,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.398454,-3.601018,2.790873,muy lejos,5.331349,muy lejos,2014-04-03,2014,4,3
2,2,26/05/2018,DE 00:00 A 00:59,SABADO,ARGANZUELA,CALLE DE EROS - CALLE DE LA ESTRELLA HADAR,0,2018/5408,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,4.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HL,DE 35 A 39 AÑOS,40.393845,-3.680282,0.948289,lejos,0.950698,lejos,2018-05-26,2018,5,26
3,3,25/02/2010,DE 14:00 A 14:59,JUEVES,CARABANCHEL,CALLE DE BELZUNEGUI - CALLE DE CARCASTILLO,0,2010/1766,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS,40.377139,-3.739303,0.984928,lejos,2.881995,muy lejos,2010-02-25,2010,2,25
4,4,17/07/2012,DE 18:00 A 18:59,MARTES,MORATALAZ,AVENIDA DEL DOCTOR GARCIA TAPIA - AUTOVIA M-40,0,2012/6707,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,3.0,COLISIÓN DOBLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 21 A 24 AÑOS,40.410009,-3.641304,1.041850,muy lejos,1.689948,muy lejos,2012-07-17,2012,7,17


In [98]:
# Cargamos calendario de festivos
df_calendario = pd.read_csv('data/CALENDARIO.csv', sep=';', decimal='.', encoding='latin-1')
df_calendario.head()

,CALE_FCALENDARIO,AÑO,MES,DIA,DIA_1,DIA_NOMBRE,DIA_HABIL
0,01-01-2010,2010,1,1,5,VIERNES,0
1,02-01-2010,2010,1,2,6,SÁBADO,1
2,03-01-2010,2010,1,3,7,DOMINGO,1
3,04-01-2010,2010,1,4,1,LUNES,1
4,05-01-2010,2010,1,5,2,MARTES,1


In [99]:
# Cambiamos el tipo de DIA_HABIL a int
df_calendario['DIA_HABIL'].astype(str).astype(int)

0       0
1       1
2       1
3       1
4       1
       ..
3281    1
3282    1
3283    1
3284    1
3285    1
Name: DIA_HABIL, Length: 3286, dtype: int32

In [100]:
# Quitamos los espacios en blanco
df_calendario['DIA_NOMBRE'] = df_calendario['DIA_NOMBRE'].str.strip()
df_calendario['DIA_NOMBRE']

0         VIERNES
1          SÁBADO
2         DOMINGO
3           LUNES
4          MARTES
          ...    
3281        LUNES
3282       MARTES
3283    MIÉRCOLES
3284       JUEVES
3285      VIERNES
Name: DIA_NOMBRE, Length: 3286, dtype: object

In [101]:
# Cambiamos los sábados y domingos a días no hábiles.
df_calendario.loc[df_calendario['DIA_NOMBRE'] == 'SÁBADO', 'DIA_HABIL'] = 0
df_calendario.loc[df_calendario['DIA_NOMBRE'] == 'DOMINGO', 'DIA_HABIL'] = 0

In [102]:
df_calendario.head()

,CALE_FCALENDARIO,AÑO,MES,DIA,DIA_1,DIA_NOMBRE,DIA_HABIL
0,01-01-2010,2010,1,1,5,VIERNES,0
1,02-01-2010,2010,1,2,6,SÁBADO,0
2,03-01-2010,2010,1,3,7,DOMINGO,0
3,04-01-2010,2010,1,4,1,LUNES,1
4,05-01-2010,2010,1,5,2,MARTES,1


In [103]:
# Cambiamos el formato a YYYY/MM/DD
df_calendario['FECHA_CON_FORMATO'] = df_calendario['CALE_FCALENDARIO'].apply(pd.to_datetime)

In [104]:
# Lo unimos en un solo dataset
df_nuevo = pd.merge(df, df_calendario,  on='FECHA_CON_FORMATO')

In [105]:
df_nuevo.head()

,Unnamed: 0,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km,Lejania ocio,Distancia_radares,Lejania_radares,FECHA_CON_FORMATO,AÑO_x,MES_x,DIA_x,CALE_FCALENDARIO,AÑO_y,MES_y,DIA_y,DIA_1,DIA_NOMBRE,DIA_HABIL
0,0,20/11/2014,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,32300,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos,0.507497,lejos,2014-11-20,2014,11,20,20-11-2014,2014,11,20,4,JUEVES,1
1,2826,20/11/2014,DE 21:00 A 21:59,JUEVES,CHAMARTIN,CALLE DEL PRINCIPE DE VERGARA - PLAZA DE CATALUÑA,0,2014/10636,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HG,DE MAS DE 74 AÑOS,40.448928,-3.677791,0.303399,Cerca,1.347168,muy lejos,2014-11-20,2014,11,20,20-11-2014,2014,11,20,4,JUEVES,1
2,4151,20/11/2014,DE 23:00 A 23:59,JUEVES,CENTRO,PLAZA DE CIBELES,2,2014/10646,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,HL,DE 25 A 29 AÑOS,40.419562,-3.692798,0.019671,Muy cerca,2.674538,muy lejos,2014-11-20,2014,11,20,20-11-2014,2014,11,20,4,JUEVES,1
3,4655,20/11/2014,DE 8:00 A 8:59,JUEVES,LATINA,CALLE VIA CARPETANA,138,2014/10766,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,CAÍDA MOTOCICLETA,TURISMO,CONDUCTOR,NO ASIGNADO,IL,DESCONOCIDA,40.748930,-3.516479,32.942337,muy lejos,32.813572,muy lejos,2014-11-20,2014,11,20,20-11-2014,2014,11,20,4,JUEVES,1
4,7274,20/11/2014,DE 19:00 A 19:59,JUEVES,CHAMBERI,CALLE DE FERNANDEZ DE LOS RIOS,76,2014/10635,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,1.0,COLISIÓN MÚLTIPLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 45 A 49 AÑOS,40.435398,-3.713425,0.018822,Muy cerca,1.836937,muy lejos,2014-11-20,2014,11,20,20-11-2014,2014,11,20,4,JUEVES,1


In [106]:
df_nuevo.columns

Index(['Unnamed: 0', 'FECHA', 'RANGO HORARIO', 'DIA SEMANA', 'DISTRITO',
       'LUGAR ACCIDENTE', 'Nº', 'Nº PARTE', 'CPFA Granizo', 'CPFA Hielo',
       'CPFA Lluvia', 'CPFA Niebla', 'CPFA Seco', 'CPFA Nieve', 'CPSV Mojada',
       'CPSV Aceite', 'CPSV Barro', 'CPSV Grava Suelta', 'CPSV Hielo',
       'CPSV Seca Y Limpia', 'Nº VICTIMAS', 'TIPO ACCIDENTE', 'Tipo Vehiculo',
       'TIPO PERSONA', 'SEXO', 'LESIVIDAD', 'Tramo Edad', 'Latitude',
       'Longitude', 'Distancia ocio Km', 'Lejania ocio', 'Distancia_radares',
       'Lejania_radares', 'FECHA_CON_FORMATO', 'AÑO_x', 'MES_x', 'DIA_x',
       'CALE_FCALENDARIO', 'AÑO_y', 'MES_y', 'DIA_y', 'DIA_1', 'DIA_NOMBRE',
       'DIA_HABIL'],
      dtype='object')

In [107]:
# Borramos las columnas que nos sobran
df_nuevo.drop(['FECHA','Unnamed: 0', 'Nº', 'Nº VICTIMAS', 'CALE_FCALENDARIO', 'AÑO_y', 'MES_y', 'DIA_1', 'DIA_y' ,'DIA_NOMBRE'], axis=1, inplace=True);

In [108]:
# Comparamos los días hábiles con los no hábiles
df_nuevo['DIA_HABIL'].value_counts()

1    150853
0     49138
Name: DIA_HABIL, dtype: int64

In [109]:
df_nuevo

,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad,Latitude,Longitude,Distancia ocio Km,Lejania ocio,Distancia_radares,Lejania_radares,FECHA_CON_FORMATO,AÑO_x,MES_x,DIA_x,DIA_HABIL
0,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos,0.507497,lejos,2014-11-20,2014,11,20,1
1,DE 21:00 A 21:59,JUEVES,CHAMARTIN,CALLE DEL PRINCIPE DE VERGARA - PLAZA DE CATALUÑA,2014/10636,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HG,DE MAS DE 74 AÑOS,40.448928,-3.677791,0.303399,Cerca,1.347168,muy lejos,2014-11-20,2014,11,20,1
2,DE 23:00 A 23:59,JUEVES,CENTRO,PLAZA DE CIBELES,2014/10646,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,HL,DE 25 A 29 AÑOS,40.419562,-3.692798,0.019671,Muy cerca,2.674538,muy lejos,2014-11-20,2014,11,20,1
3,DE 8:00 A 8:59,JUEVES,LATINA,CALLE VIA CARPETANA,2014/10766,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,CAÍDA MOTOCICLETA,TURISMO,CONDUCTOR,NO ASIGNADO,IL,DESCONOCIDA,40.748930,-3.516479,32.942337,muy lejos,32.813572,muy lejos,2014-11-20,2014,11,20,1
4,DE 19:00 A 19:59,JUEVES,CHAMBERI,CALLE DE FERNANDEZ DE LOS RIOS,2014/10635,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,COLISIÓN MÚLTIPLE,FURGONETA,CONDUCTOR,HOMBRE,IL,DE 45 A 49 AÑOS,40.435398,-3.713425,0.018822,Muy cerca,1.836937,muy lejos,2014-11-20,2014,11,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,DE 21:00 A 21:59,JUEVES,SALAMANCA,CALLE DEL PRINCIPE DE VERGARA - CALLE DE GOYA,2010/5385,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 18 A 20 AÑOS,40.425010,-3.680107,0.171356,Muy cerca,1.695370,muy lejos,2010-03-06,2010,3,6,1
199987,DE 14:00 A 14:59,JUEVES,FUENCARRAL-EL PARDO,AUTOVIA M-30 CALZADA 1 KM.,2010/5031,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,COLISIÓN MÚLTIPLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 60 A 64 AÑOS,40.490213,-3.698348,0.327146,Cerca,1.538157,muy lejos,2010-03-06,2010,3,6,1
199988,DE 13:00 A 13:59,JUEVES,CHAMARTIN,AVENIDA DE ASTURIAS - CALLE DE SOROLLA,2010/5003,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,ATROPELLO,TURISMO,CONDUCTOR,HOMBRE,IL,DE 45 A 49 AÑOS,40.470667,-3.696902,0.812785,lejos,1.461029,muy lejos,2010-03-06,2010,3,6,1
199989,DE 15:00 A 15:59,JUEVES,SALAMANCA,AVENIDA DE BRASILIA,2010/5386,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,COLISIÓN DOBLE,TURISMO,CONDUCTOR,MUJER,IL,DE 35 A 39 AÑOS,40.440709,-3.661033,0.456987,Cerca,1.617061,muy lejos,2010-03-06,2010,3,6,1


In [110]:
# Vamos a renombrar los nombres de las columnas
df_nuevo.columns

Index(['RANGO HORARIO', 'DIA SEMANA', 'DISTRITO', 'LUGAR ACCIDENTE',
       'Nº PARTE', 'CPFA Granizo', 'CPFA Hielo', 'CPFA Lluvia', 'CPFA Niebla',
       'CPFA Seco', 'CPFA Nieve', 'CPSV Mojada', 'CPSV Aceite', 'CPSV Barro',
       'CPSV Grava Suelta', 'CPSV Hielo', 'CPSV Seca Y Limpia',
       'TIPO ACCIDENTE', 'Tipo Vehiculo', 'TIPO PERSONA', 'SEXO', 'LESIVIDAD',
       'Tramo Edad', 'Latitude', 'Longitude', 'Distancia ocio Km',
       'Lejania ocio', 'Distancia_radares', 'Lejania_radares',
       'FECHA_CON_FORMATO', 'AÑO_x', 'MES_x', 'DIA_x', 'DIA_HABIL'],
      dtype='object')

In [111]:
df_nuevo.columns = ['Rango_horario', 'Dia_semana', 'Distrito', 'Lugar_accidente',
       'Parte', 'Granizo', 'Hielo', 'Lluvia', 'Niebla',
       'Seco', 'Nieve', 'Mojada', 'Aceite', 'Barro',
       'Gravilla', 'Hielo', 'Seca_limpia','Tipo_accidente', 
       'Tipo_vehiculo', 'Tipo_persona', 'Sexo', 'Lesividad',
       'Tramo_edad', 'Latitud', 'Longitud', 'Distancia_ocio_km',
       'Lejania_ocio', 'Distancia_radares_km', 'Lejania_radares',
       'Fecha_formato', 'Year', 'Mes', 'Dia', 'Dia_habil']

In [112]:
df_nuevo.head(2)

,Rango_horario,Dia_semana,Distrito,Lugar_accidente,Parte,Granizo,Hielo,Lluvia,Niebla,Seco,Nieve,Mojada,Aceite,Barro,Gravilla,Hielo,Seca_limpia,Tipo_accidente,Tipo_vehiculo,Tipo_persona,Sexo,Lesividad,Tramo_edad,Latitud,Longitud,Distancia_ocio_km,Lejania_ocio,Distancia_radares_km,Lejania_radares,Fecha_formato,Year,Mes,Dia,Dia_habil
0,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,2014/10641,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,HL,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos,0.507497,lejos,2014-11-20,2014,11,20,1
1,DE 21:00 A 21:59,JUEVES,CHAMARTIN,CALLE DEL PRINCIPE DE VERGARA - PLAZA DE CATALUÑA,2014/10636,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,SI,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,HG,DE MAS DE 74 AÑOS,40.448928,-3.677791,0.303399,Cerca,1.347168,muy lejos,2014-11-20,2014,11,20,1


In [113]:
# Nos llevamos la variable objetivo Lesividad a la primera posición
cols = df_nuevo.columns.tolist()
cols.insert(0, cols.pop(cols.index('Lesividad')))
df_nuevo = df_nuevo[cols]
df_nuevo.head()

,Lesividad,Rango_horario,Dia_semana,Distrito,Lugar_accidente,Parte,Granizo,Hielo,Hielo,Lluvia,Niebla,Seco,Nieve,Mojada,Aceite,Barro,Gravilla,Hielo,Hielo,Seca_limpia,Tipo_accidente,Tipo_vehiculo,Tipo_persona,Sexo,Tramo_edad,Latitud,Longitud,Distancia_ocio_km,Lejania_ocio,Distancia_radares_km,Lejania_radares,Fecha_formato,Year,Mes,Dia,Dia_habil
0,HL,DE 9:00 A 9:59,JUEVES,CHAMARTIN,AUTOVIA M-30 CALZADA 2 KM.,2014/10641,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO,SI,COLISIÓN DOBLE,MOTOCICLETA,CONDUCTOR,HOMBRE,DE 21 A 24 AÑOS,40.483873,-3.673592,1.508801,muy lejos,0.507497,lejos,2014-11-20,2014,11,20,1
1,HG,DE 21:00 A 21:59,JUEVES,CHAMARTIN,CALLE DEL PRINCIPE DE VERGARA - PLAZA DE CATALUÑA,2014/10636,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO,SI,ATROPELLO,NO ASIGNADO,PEATON,HOMBRE,DE MAS DE 74 AÑOS,40.448928,-3.677791,0.303399,Cerca,1.347168,muy lejos,2014-11-20,2014,11,20,1
2,HL,DE 23:00 A 23:59,JUEVES,CENTRO,PLAZA DE CIBELES,2014/10646,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO,SI,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,DE 25 A 29 AÑOS,40.419562,-3.692798,0.019671,Muy cerca,2.674538,muy lejos,2014-11-20,2014,11,20,1
3,IL,DE 8:00 A 8:59,JUEVES,LATINA,CALLE VIA CARPETANA,2014/10766,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO,SI,CAÍDA MOTOCICLETA,TURISMO,CONDUCTOR,NO ASIGNADO,DESCONOCIDA,40.748930,-3.516479,32.942337,muy lejos,32.813572,muy lejos,2014-11-20,2014,11,20,1
4,IL,DE 19:00 A 19:59,JUEVES,CHAMBERI,CALLE DE FERNANDEZ DE LOS RIOS,2014/10635,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO,SI,COLISIÓN MÚLTIPLE,FURGONETA,CONDUCTOR,HOMBRE,DE 45 A 49 AÑOS,40.435398,-3.713425,0.018822,Muy cerca,1.836937,muy lejos,2014-11-20,2014,11,20,1


In [114]:
df_nuevo.to_csv('./Accidentefinal.csv',sep=';', decimal='.')